In [114]:
import sys, os
from os.path import join, dirname, abspath
import matplotlib.pyplot as plt
from matplotlib.pyplot import Figure, Axes
import numpy as np
import networkx as nx
import matplotlib.animation as animation
from string import ascii_uppercase
plt.rcParams.update({
    "text.usetex": False,
    "ytick.minor.visible":True,
    "xtick.minor.visible":True,
    'xtick.direction': "in",
    'ytick.direction': "in"
})
outdir = "hw6_out"
os.makedirs(outdir,exist_ok=True)
def out(fname): return join(outdir,fname)
def savefig(plot_name): 
    plt.savefig(out(plot_name),bbox_inches="tight",dpi=250)
import pandas as pd
from numpy.linalg import matrix_power, eig

def arr_to_latex(M):
    return '$$\n' + r'\begin{bmatrix}' + '\n' + (r'\\' + '\n').join('&'.join(str(x) for x in row) for row in M) + '\n' + r'\end{bmatrix}' + '\n' +'$$'

def vec_to_latex(x,round=3):
    return '$$\n' + r'\begin{bmatrix}' + '\n' + (r' \\ ').join(str(np.round(v,round)) for v in x) + '\n' + r'\end{bmatrix}' + '\n' +'$$'

# Question 1: Logistic ODE

## Part a: Symbolic

In [115]:
from sympy import Function, dsolve, Derivative, checkodesol, symbols, lambdify, Equality
from sympy.abc import x

In [ ]:
x = Function("x")
r, t, k, x0 = symbols("r, t, k, x0")
eq = Equality(Derivative(x(t),t),r*x(t)*(1-x(t)/k))
print(eq._repr_latex_())
eq

In [ ]:
result = dsolve(eq,x(t),ics={x(0):x0}).simplify()
print(result._repr_latex_())
result

## Part b: Plot

In [118]:
K = 100
R = 0.75
X0 = 10
t0, tf = 0, 10

In [ ]:
subst = result.subs({k:K,r:R,x0:X0})
xt = lambdify(t,subst.rhs,"numpy")
t_arr = np.linspace(t0,tf,num=1000)
plt.plot(t_arr,xt(t_arr))
plt.xlabel("t")
plt.ylabel("x(t)")
plt.title("Symbolic Solution for Logistic Eq")
savefig("q1_symbolic_plot.png")

## Part c: numerically solve

In [120]:
from scipy.integrate import solve_ivp

In [ ]:
def logistic(t,y): return R*y*(1-y/K)
sol = solve_ivp(logistic,t_span=(t0,tf),t_eval=t_arr,y0=[X0])
sol

In [ ]:
plt.plot(t_arr,xt(t_arr),label="Symbolic")
plt.xlabel("t")
plt.ylabel("x(t)")
plt.title("Symbolic vs Numeric Solution for Logistic Eq")
plt.plot(sol.t,sol.y[0],label="RK45")
plt.legend()
savefig("q1_numeric_plot.png")

In [ ]:
xt(t_arr)-sol.y[0]

In [ ]:
plt.plot(t_arr,xt(t_arr)-sol.y[0])
plt.title("Deviation of RK45 from Symbolic")
plt.xlabel('$t$')
plt.ylabel('$x(t)_{sym}-x(t)_{RK45}$')
savefig("q1_absolute_deviation.png")

In [ ]:
plt.plot(t_arr,(xt(t_arr)-sol.y[0])/xt(t_arr))
plt.title("Fractional Deviation of RK45 from Symbolic")
plt.xlabel('$t$')
plt.ylabel('$(x(t)_{sym}-x(t)_{RK45})/x(t)_{sym}$')
savefig("q1_fractional_deviation.png")

# Problem 3: SIR

In [175]:
S_ = 'tab:blue'
I_ = 'tab:red'
R_ = 'tab:orange'

In [176]:
S0 = 10
I0 = 1
R0 = 0

t0, tf = 0, 100
t_arr = np.linspace(t0,tf,10000)

y0 = [S0,I0,R0]

In [177]:
def SIR(beta,gamma):
    def _sir(t, sir):
        return [-beta * sir[0] * sir[1], beta*sir[0]*sir[1]-gamma*sir[1], gamma*sir[1]]
    return _sir

In [ ]:
sir(1,y0)

In [ ]:
beta, gamma = 0.01,0.02
sir=SIR(beta,gamma)
sol = solve_ivp(sir,(t0,tf),y0,t_eval=t_arr)
plt.plot(t_arr, sol.y[0],label="S",color=S_)
plt.plot(t_arr, sol.y[1],label="I",color=I_)
plt.plot(t_arr, sol.y[2],label="R",color=R_)

plt.legend()
plt.title(r"SIR: $\beta="+f"{beta}"+r", \gamma="+f"{gamma}$")

In [ ]:
fig,ax = plt.subplots()

betas = [0.01,0.0105,0.011]

beta, gamma = betas[0],0.02
sir=SIR(beta,gamma)
sol = solve_ivp(sir,(t0,tf),y0,t_eval=t_arr)
ax.plot(t_arr, sol.y[0],label="S",color=S_)
ax.plot(t_arr, sol.y[1],label="I",color=I_)
ax.plot(t_arr, sol.y[2],label="R",color=R_)
l1 = ax.legend()

beta, gamma = betas[1],0.02
sir=SIR(beta,gamma)
sol = solve_ivp(sir,(t0,tf),y0,t_eval=t_arr)
ax.plot(t_arr, sol.y[0],color=S_,linestyle="dashed")
ax.plot(t_arr, sol.y[1],color=I_,linestyle="dashed")
ax.plot(t_arr, sol.y[2],color=R_,linestyle="dashed")

beta, gamma = betas[2],0.02
sir=SIR(beta,gamma)
sol = solve_ivp(sir,(t0,tf),y0,t_eval=t_arr)
ax.plot(t_arr, sol.y[0],color=S_,linestyle="dotted")
ax.plot(t_arr, sol.y[1],color=I_,linestyle="dotted")
ax.plot(t_arr, sol.y[2],color=R_,linestyle="dotted")

ax2 = ax.twinx()
ax2.set_visible(False)
ax2.plot(1,1,color="black",label=r"$\beta="+f"{betas[0]}$")
ax2.plot(1,1,color="black",label=r"$\beta="+f"{betas[1]}$",linestyle="dashed")
ax2.plot(1,1,color="black",label=r"$\beta="+f"{betas[2]}$",linestyle="dotted")
ax2.set_visible(True)
ax2.set_axis_off()

l2 = ax2.legend(ncols=len(betas),loc="upper center")
ax.add_artist(l1)

ax.set_ylabel("Class Population")
ax.set_xlabel("$t$")
ax.set_title(r"SIR with variable $\beta, \gamma="+f"{gamma}$")
savefig("variable_beta.png")

In [ ]:
fig,ax = plt.subplots()

gammas = [0.02,0.0195,0.021]

beta, gamma = 0.01,gammas[0]
sir=SIR(beta,gamma)
sol = solve_ivp(sir,(t0,tf),y0,t_eval=t_arr)
ax.plot(t_arr, sol.y[0],label="S",color=S_)
ax.plot(t_arr, sol.y[1],label="I",color=I_)
ax.plot(t_arr, sol.y[2],label="R",color=R_)
l1 = ax.legend()

beta, gamma = 0.01,gammas[1]
sir=SIR(beta,gamma)
sol = solve_ivp(sir,(t0,tf),y0,t_eval=t_arr)
ax.plot(t_arr, sol.y[0],color=S_,linestyle="dashed")
ax.plot(t_arr, sol.y[1],color=I_,linestyle="dashed")
ax.plot(t_arr, sol.y[2],color=R_,linestyle="dashed")

beta, gamma = 0.01,gammas[2]
sir=SIR(beta,gamma)
sol = solve_ivp(sir,(t0,tf),y0,t_eval=t_arr)
ax.plot(t_arr, sol.y[0],color=S_,linestyle="dotted")
ax.plot(t_arr, sol.y[1],color=I_,linestyle="dotted")
ax.plot(t_arr, sol.y[2],color=R_,linestyle="dotted")

ax2 = ax.twinx()
ax2.set_visible(False)
ax2.plot(1,1,color="black",label=r"$\gamma="+f"{gammas[0]}$")
ax2.plot(1,1,color="black",label=r"$\gamma="+f"{gammas[1]}$",linestyle="dashed")
ax2.plot(1,1,color="black",label=r"$\gamma="+f"{gammas[2]}$",linestyle="dotted")
ax2.set_visible(True)
ax2.set_axis_off()

l2 = ax2.legend(ncols=len(betas),loc="upper center")
ax.add_artist(l1)

ax.set_ylabel("Class Population")
ax.set_xlabel("$t$")
ax.set_title(r"SIR with variable $\gamma, \beta="+f"{beta}$")
savefig("variable_gamma.png")

In [ ]:
beta,gamma = 0.01,0.02
fig,ax = plt.subplots()

y0s=[(10,1,0),(9,2,0),(8,1,2)]

sir=SIR(beta,gamma)
y0 = y0s[0]
sol = solve_ivp(sir,(t0,tf),y0,t_eval=t_arr)
ax.plot(t_arr,sol.y[0],label="S",color=S_)
ax.plot(t_arr,sol.y[1],label="I",color=I_)
ax.plot(t_arr,sol.y[2],label="R",color=R_)
l1 = ax.legend()

y0 = y0s[1]
sol = solve_ivp(sir,(t0,tf),y0,t_eval=t_arr)
ax.plot(t_arr,sol.y[0],color=S_,linestyle="dashed")
ax.plot(t_arr,sol.y[1],color=I_,linestyle="dashed")
ax.plot(t_arr,sol.y[2],color=R_,linestyle="dashed")

y0 = y0s[2]
sol = solve_ivp(sir,(t0,tf),y0,t_eval=t_arr)
ax.plot(t_arr,sol.y[0],color=S_,linestyle="dotted")
ax.plot(t_arr,sol.y[1],color=I_,linestyle="dotted")
ax.plot(t_arr,sol.y[2],color=R_,linestyle="dotted")

ax2 = ax.twinx()
ax2.set_visible(False)
ax2.plot(1,1,color="black",label=f"$S_0={y0s[0][0]}, I_0={y0s[0][1]}, R_0={y0s[0][2]}$")
ax2.plot(1,1,color="black",label=f"$S_0={y0s[1][0]}, I_0={y0s[1][1]}, R_0={y0s[1][2]}$",linestyle="dashed")
ax2.plot(1,1,color="black",label=f"$S_0={y0s[2][0]}, I_0={y0s[2][1]}, R_0={y0s[2][2]}$",linestyle="dotted")
ax2.set_visible(True)
ax2.set_axis_off()

l2 = ax2.legend(loc="upper center")
ax.add_artist(l1)

ax.set_ylabel("Class Population")
ax.set_xlabel("$t$")
ax.set_title(r"SIR with $\gamma=" + f"{gamma}"+r", \beta="+f"{beta}$, variable ICs")
savefig("variable_ics.png")